Project

In [1]:
# Import necessary libraries
# May takes few minutes
import os
import re
import ast
from pathlib import Path

import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.inspection import permutation_importance

In [2]:
# Step 4.1 — Setup paths, load polygons, ensure CRS, and quick QC plot

data_dir = Path('original_data')
reports_dir = Path('reports')
fig_dir = reports_dir / 'figures'
reports_dir.mkdir(parents=True, exist_ok=True)
fig_dir.mkdir(parents=True, exist_ok=True)

van_geo_path = data_dir / 'van_neighbourhoods.geojson'
vic_geo_path = data_dir / 'vic_neighbourhoods.geojson'

# Helpers
EPSG_WGS84 = 4326

def ensure_wgs84(gdf):
    if gdf.crs is None:
        return gdf.set_crs(EPSG_WGS84)
    try:
        epsg = gdf.crs.to_epsg()
    except Exception:
        epsg = None
    return gdf if epsg == EPSG_WGS84 else gdf.to_crs(EPSG_WGS84)

# Load geospatial data
van = gpd.read_file(van_geo_path)
van = ensure_wgs84(van)

vic = gpd.read_file(vic_geo_path)
vic = ensure_wgs84(vic)

print('Vancouver polygons:', len(van), '| CRS:', van.crs)
print('Victoria polygons:', len(vic), '| CRS:', vic.crs)

# Quick QC boundary plot
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
van.boundary.plot(ax=ax[0], color='steelblue', linewidth=0.8)
ax[0].set_title('Vancouver neighbourhoods')
ax[0].axis('off')

vic.boundary.plot(ax=ax[1], color='darkorange', linewidth=0.8)
ax[1].set_title('Victoria neighbourhoods')
ax[1].axis('off')

out_path = fig_dir / 'boundaries_quickcheck_step4.png'
fig.tight_layout()
fig.savefig(out_path, dpi=150)
plt.close(fig)
print(f'Saved QC plot to: {out_path}')

Vancouver polygons: 24 | CRS: EPSG:4326
Victoria polygons: 29 | CRS: EPSG:4326
Saved QC plot to: reports\figures\boundaries_quickcheck_step4.png


In [3]:
# Step 4.2 — Load listings (.csv.gz) and inspect

data_dir = Path('original_data')

# Auto-discover input files that look like listings exports for 2025
listing_paths = sorted(data_dir.glob('*listings*2025*.csv.gz'))
listing_files = [p.name for p in listing_paths]
if not listing_files:
    # Fallback to the known four if glob finds nothing
    listing_files = [
        'van_listings_April2025.csv.gz',
        'van_listings_June2025.csv.gz',
        'vic_listings_March2025.csv.gz',
        'vic_listings_May2025.csv.gz'
    ]
    listing_paths = [data_dir / f for f in listing_files]

loaded = {}
for fpath in listing_paths:
    df_l = pd.read_csv(fpath, compression='gzip', low_memory=False)
    loaded[fpath.name] = df_l
    print(f"Loaded {fpath.name}: shape={df_l.shape}")

# schema candidates
candidates = {
    # core fields used in Step 4.3
    'price': ['price'],
    'neighbourhood': ['neighbourhood_cleansed', 'neighbourhood'],
    'date': ['last_scraped', 'calendar_last_scraped'],
    'latitude': ['latitude'],
    'longitude': ['longitude'],
    'estimated_revenue_l365d': ['estimated_revenue_l365d'],
    'estimated_occupancy_l365d': ['estimated_occupancy_l365d'],
    'availability_365': ['availability_365'],
    'availability_30': ['availability_30'],
    'reviews_per_month': ['reviews_per_month'],
    'number_of_reviews': ['number_of_reviews'],
    'amenities': ['amenities'],
    'property_type': ['property_type'],
    'room_type': ['room_type'],
    'accommodates': ['accommodates'],
    'bathrooms': ['bathrooms'],
    'bathrooms_text': ['bathrooms_text'],
    'bedrooms': ['bedrooms'],
    'beds': ['beds'],
    'host_is_superhost': ['host_is_superhost'],
    'host_since': ['host_since'],
    'license': ['license'],
    'calculated_host_listings_count': ['calculated_host_listings_count'],
    'minimum_nights': ['minimum_nights'],
    'maximum_nights': ['maximum_nights'],
}

# returns the original column name
def pick_exact(df, options):
    lower_to_orig = {c.lower(): c for c in df.columns}
    for opt in options:
        c = lower_to_orig.get(opt.lower())
        if c:
            return c
    return None

schema_map = {}
for fname, df_l in loaded.items():
    mapping = {k: pick_exact(df_l, v) for k, v in candidates.items()}
    schema_map[fname] = mapping

# schema_map # check schema mappings

Loaded van_listings_April2025.csv.gz: shape=(5460, 79)
Loaded van_listings_June2025.csv.gz: shape=(5090, 79)
Loaded vic_listings_March2025.csv.gz: shape=(3712, 79)
Loaded vic_listings_May2025.csv.gz: shape=(3478, 79)


In [4]:
# Step 4.3 — Unify listings schema (price, city, month, neighbourhood)
import re
import numpy as np

month_map = {
    'January': '01', 'February': '02', 'March': '03', 'April': '04',
    'May': '05', 'June': '06', 'July': '07', 'August': '08',
    'September': '09', 'October': '10', 'November': '11', 'December': '12'
}

city_from_fname = lambda n: 'Vancouver' if n.lower().startswith('van_') else ('Victoria' if n.lower().startswith('vic_') else None)

# price parser robust to $ and commas
def parse_price(val):
    if pd.isna(val):
        return np.nan
    if isinstance(val, (int, float)):
        return float(val)
    s = str(val)
    s = re.sub(r'[^0-9.]+', '', s)
    try:
        return float(s) if s else np.nan
    except ValueError:
        return np.nan

# Helper to clean neighbourhood strings
def clean_neigh(s: pd.Series) -> pd.Series:
    if s is None:
        return pd.Series(np.nan, index=range(0))
    x = (s.astype(str)
           .str.strip()
           .str.replace(r'\s+', ' ', regex=True))
    # Remove known placeholder labels (both British/US spelling)
    x = x.str.replace(r'(?i)\bneighbourhood highlights\b', '', regex=True)
    x = x.str.replace(r'(?i)\bneighborhood highlights\b', '', regex=True)
    # If value becomes empty or 'nan', set NaN
    x = x.replace({'': np.nan, 'nan': np.nan, 'None': np.nan})
    # Remove leading/trailing punctuation
    x = x.str.replace(r'^[^A-Za-z0-9]+|[^A-Za-z0-9]+$', '', regex=True)
    # Title-case
    x = x.str.title()
    return x

# Fields try to pass through when present
pass_through_fields = [
    'estimated_revenue_l365d','estimated_occupancy_l365d','availability_365','availability_30',
    'reviews_per_month','number_of_reviews','amenities','property_type','room_type','accommodates',
    'bathrooms','bathrooms_text','bedrooms','beds','host_is_superhost','host_since','license',
    'calculated_host_listings_count','minimum_nights','maximum_nights'
]

frames = []
for fname, df_l in loaded.items():
    city = city_from_fname(fname)
    m = re.search(r'(January|February|March|April|May|June|July|August|September|October|November|December)(\d{4})', fname)
    if m:
        month_str = f"{m.group(2)}-{month_map[m.group(1)]}"
    else:
        # fallback to last_scraped's month
        month_col = schema_map[fname].get('date')
        month_str = pd.to_datetime(df_l[month_col], errors='coerce').dt.strftime('%Y-%m').iloc[0] if month_col else np.nan

    # choose columns from schema map
    price_col = schema_map[fname].get('price')
    neigh_col = schema_map[fname].get('neighbourhood')
    date_col = schema_map[fname].get('date')
    lat_col = schema_map[fname].get('latitude')
    lon_col = schema_map[fname].get('longitude')

    out = pd.DataFrame()
    out['listing_id'] = df_l['id'] if 'id' in df_l.columns else df_l.index
    out['city'] = city
    out['month'] = month_str

    if neigh_col:
        out['neighbourhood'] = clean_neigh(df_l[neigh_col])
    else:
        out['neighbourhood'] = np.nan

    if price_col:
        out['price'] = df_l[price_col].apply(parse_price)
    else:
        out['price'] = np.nan
    if date_col:
        out['last_scraped'] = pd.to_datetime(df_l[date_col], errors='coerce')

    # optional coords using detected exact columns
    if lat_col and lat_col in df_l.columns:
        out['latitude'] = pd.to_numeric(df_l[lat_col], errors='coerce')
    if lon_col and lon_col in df_l.columns:
        out['longitude'] = pd.to_numeric(df_l[lon_col], errors='coerce')

    # pass-through fields when available
    mapping = schema_map[fname]
    for key in pass_through_fields:
        src = mapping.get(key)
        if src and src in df_l.columns:
            out[key] = df_l[src]

    frames.append(out)

listings = pd.concat(frames, ignore_index=True)
print('Unified listings shape:', listings.shape)
print(listings[['city','month']].value_counts().sort_index())
print('\nColumns:', listings.columns.tolist())
listings.head(5)

Unified listings shape: (17740, 28)
city       month  
Vancouver  2025-04    5460
           2025-06    5090
Victoria   2025-03    3712
           2025-05    3478
Name: count, dtype: int64

Columns: ['listing_id', 'city', 'month', 'neighbourhood', 'price', 'last_scraped', 'latitude', 'longitude', 'estimated_revenue_l365d', 'estimated_occupancy_l365d', 'availability_365', 'availability_30', 'reviews_per_month', 'number_of_reviews', 'amenities', 'property_type', 'room_type', 'accommodates', 'bathrooms', 'bathrooms_text', 'bedrooms', 'beds', 'host_is_superhost', 'host_since', 'license', 'calculated_host_listings_count', 'minimum_nights', 'maximum_nights']


,listing_id,city,month,neighbourhood,price,last_scraped,latitude,longitude,estimated_revenue_l365d,estimated_occupancy_l365d,...,bathrooms,bathrooms_text,bedrooms,beds,host_is_superhost,host_since,license,calculated_host_listings_count,minimum_nights,maximum_nights
0,13188,Vancouver,2025-04,Riley Park,136.0,2025-04-10,49.247730,-123.105090,34680.0,255,...,1.0,1 bath,0.0,2.0,t,2009-11-04,25-156058,1,2,180
1,13358,Vancouver,2025-04,Downtown,198.0,2025-04-10,49.281174,-123.125931,50490.0,255,...,1.0,1 bath,1.0,1.0,t,2009-11-07,25-157257,1,1,1125
2,16254,Vancouver,2025-04,Hastings-Sunrise,680.0,2025-04-10,49.277210,-123.040860,0.0,0,...,1.0,1 bath,2.0,3.0,f,2009-12-15,NaN,1,5,31
3,16611,Vancouver,2025-04,Grandview-Woodland,NaN,2025-04-10,49.263390,-123.071450,NaN,0,...,NaN,1 bath,3.0,NaN,f,2009-11-29,NaN,5,365,365
4,17765,Vancouver,2025-04,Mount Pleasant,NaN,2025-04-10,49.261320,-123.108450,NaN,0,...,NaN,1 bath,1.0,NaN,f,2010-01-07,21-156705,1,5,60


In [5]:
# Step 4.4 — Project polygons to metric CRS and compute area (km²)

EPSG_METRIC = 26910  # UTM Zone 10N (BC west)

van_prj = van.to_crs(EPSG_METRIC)
vic_prj = vic.to_crs(EPSG_METRIC)

van_prj['area_km2'] = van_prj.geometry.area / 1e6
vic_prj['area_km2'] = vic_prj.geometry.area / 1e6

print('Vancouver area stats (km²):', van_prj['area_km2'].describe().round(3).to_dict())
print('Victoria area stats (km²):', vic_prj['area_km2'].describe().round(3).to_dict())

# persist minimal cleaned polygons
cleaned_dir = Path('cleaned_data')
cleaned_dir.mkdir(exist_ok=True)
van_out = cleaned_dir / 'van_neighbourhoods_clean.geojson'
vic_out = cleaned_dir / 'vic_neighbourhoods_clean.geojson'

van_prj[['neighbourhood','neighbourhood_group','area_km2','geometry']].to_file(van_out, driver='GeoJSON')
vic_prj[['neighbourhood','neighbourhood_group','area_km2','geometry']].to_file(vic_out, driver='GeoJSON')
print('Saved cleaned polygons to:', van_out, 'and', vic_out)

Vancouver area stats (km²): {'count': 24.0, 'mean': 4.678, 'std': 2.162, 'min': 0.639, '25%': 3.262, '50%': 4.467, '75%': 6.29, 'max': 8.581}
Victoria area stats (km²): {'count': 29.0, 'mean': 167.061, 'std': 565.565, 'min': 0.24, '25%': 1.579, '50%': 7.334, '75%': 46.992, 'max': 2901.212}
Saved cleaned polygons to: cleaned_data\van_neighbourhoods_clean.geojson and cleaned_data\vic_neighbourhoods_clean.geojson


In [6]:
# Step 4.5 — Clean listings (price/date/coords) and persist

assert 'listings' in globals(), 'Run Step 4.3 first to build listings'

clean = listings.copy()

# Coerce date
if 'last_scraped' in clean.columns:
    clean['last_scraped'] = pd.to_datetime(clean['last_scraped'], errors='coerce')

# Filter price outliers (keep reasonable nightly range)
low, high = 20, 10000
before = len(clean)
clean = clean[clean['price'].between(low, high)]
print(f'Filtered by price [{low},{high}] -> {before} -> {len(clean)} rows')

# Standardize neighbourhood text and remove placeholders
clean['neighbourhood'] = (clean['neighbourhood']
                          .astype(str)
                          .str.strip()
                          .str.replace('\n', ' ', regex=False)
                          .str.replace('\r', ' ', regex=False)
                          .str.replace(r'\s+', ' ', regex=True))
clean['neighbourhood'] = clean['neighbourhood'].str.replace(r'(?i)\bneighbourhood highlights\b', '', regex=True)
clean['neighbourhood'] = clean['neighbourhood'].str.replace(r'(?i)\bneighborhood highlights\b', '', regex=True)
# Normalize empties to NaN
clean['neighbourhood'] = clean['neighbourhood'].replace({'': np.nan, 'nan': np.nan, 'None': np.nan})
# Title-case valid values
clean.loc[clean['neighbourhood'].notna(), 'neighbourhood'] = clean.loc[clean['neighbourhood'].notna(), 'neighbourhood'].str.title()

# Drop rows without neighbourhood or coordinates (for spatial join later)
before = len(clean)
if {'latitude','longitude'}.issubset(clean.columns):
    clean = clean.dropna(subset=['latitude','longitude'])
clean = clean.dropna(subset=['neighbourhood'])

# Basic stats
print('Cleaned counts by city-month:')
print(clean[['city','month']].value_counts().sort_index())
print('Neighbourhood unique count:', clean['neighbourhood'].nunique())

# Persist
out_csv = Path('cleaned_data') / 'listings_unified_clean.csv'
out_csv.parent.mkdir(exist_ok=True)
clean.to_csv(out_csv, index=False)
print(f'Saved cleaned listings to: {out_csv}')

Filtered by price [20,10000] -> 17740 -> 14479 rows
Cleaned counts by city-month:
city       month  
Vancouver  2025-04    4312
           2025-06    4236
Victoria   2025-03    3119
           2025-05    2812
Name: count, dtype: int64
Neighbourhood unique count: 51
Saved cleaned listings to: cleaned_data\listings_unified_clean.csv


## RQ1 — What predicts annual Airbnb revenue? (Listing-level modeling)
We build a listing-level dataset, engineer features, derive an annual revenue target from nightly price and an occupancy proxy, and fit models to quantify predictors.

In [7]:
# RQ1 — Build features and target; define train/test split
assert 'clean' in globals(), 'Run Step 4.5 first to build clean listings DataFrame'
df0 = pd.read_csv('cleaned_data/listings_unified_clean.csv', low_memory=False)
# 2) Minimal feature engineering
# amenities_count
if 'amenities' in df0.columns:
    def amenity_count(x):
        if pd.isna(x):
            return np.nan
        s = str(x)
        try:
            v = ast.literal_eval(s)
            if isinstance(v, (list, tuple, set)):
                return len(v)
        except Exception:
            pass
        # fallback: comma-separated string
        return int(len([t for t in s.split(',') if t.strip()]))
    df0['amenities_count'] = df0['amenities'].apply(amenity_count)

# Normalize host_is_superhost to string category
if 'host_is_superhost' in df0.columns:
    df0['host_is_superhost'] = (
        df0['host_is_superhost']
        .astype(str)
        .str.strip()
        .str.lower()
        .map({'t':'true','true':'true','f':'false','false':'false'})
        .fillna('unknown')
    )

# 3) Create target y (annual revenue)
# Preferred: estimated_revenue_l365d
if 'estimated_revenue_l365d' in df0.columns and df0['estimated_revenue_l365d'].notna().mean() > 0.3:
    y = pd.to_numeric(df0['estimated_revenue_l365d'], errors='coerce')
else:
    # Fallback: price * occupancy * 365
    price = pd.to_numeric(df0.get('price', np.nan), errors='coerce')
    occ = None
    if 'estimated_occupancy_l365d' in df0.columns:
        occ = pd.to_numeric(df0['estimated_occupancy_l365d'], errors='coerce')
        if occ.dropna().max() > 1.5:
            occ = occ / 100.0  # convert fraction
    elif 'availability_365' in df0.columns:
        avail = pd.to_numeric(df0['availability_365'], errors='coerce')
        occ = 1.0 - (avail / 365.0)
    else:
        occ = pd.Series(0.5, index=df0.index)  
    y = price * occ * 365.0

# 4) Select features
numeric_features = [
    'price','accommodates','bathrooms','bedrooms','beds',
    'reviews_per_month','number_of_reviews','availability_365','availability_30',
    'calculated_host_listings_count','minimum_nights','maximum_nights','amenities_count'
]

categorical_features = [
    'property_type','room_type','host_is_superhost','license','city'
]

# keep only columns that exist
numeric_features = [c for c in numeric_features if c in df0.columns]
categorical_features = [c for c in categorical_features if c in df0.columns]

# Build feature frame
features_df = df0[numeric_features + categorical_features].copy()

# Ensure categorical fields are strings with no NaN (avoid OHE issues)
for c in categorical_features:
    features_df[c] = features_df[c].astype('string').fillna('Unknown')

# 5) Drop rows with missing target and align
mask = y.notna()
X = features_df[mask]
y = y[mask]

# 6) Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42
)

print('Prepared features for RQ1:')
print('  rows:', len(X))
print('  numeric features:', numeric_features)
print('  categorical features:', categorical_features)

Prepared features for RQ1:
  rows: 14479
  numeric features: ['price', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'reviews_per_month', 'number_of_reviews', 'availability_365', 'availability_30', 'calculated_host_listings_count', 'minimum_nights', 'maximum_nights', 'amenities_count']
  categorical features: ['property_type', 'room_type', 'host_is_superhost', 'license', 'city']


In [8]:
# RQ1 continued: modeling — KNN and RandomForest top-predictor extraction
# Warning: This cell contains training large models and may take several minutes to run.
# Expect variables from previous cell: features_df, y, X_train, X_test, y_train, y_test, numeric_features, categorical_features

assert 'X_train' in globals(), 'Run the feature preparation steps first to define X_train, y_train, etc.'

num_feats = [f for f in numeric_features if f in X_train.columns]
cat_feats = [f for f in categorical_features if f in X_train.columns]

# Reference: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html
# Limit OHE categories by grouping infrequent levels to reduce dimensionality
ohe = OneHotEncoder(handle_unknown='infrequent_if_exist', min_frequency=0.01)

# Preprocessors
prep_knn = ColumnTransformer([
    ('num', Pipeline([('imp', SimpleImputer(strategy='median')), ('sc', StandardScaler())]), num_feats),
    ('cat', ohe, cat_feats),
], remainder='drop')

prep_rf = ColumnTransformer([
    ('num', SimpleImputer(strategy='median'), num_feats),
    ('cat', ohe, cat_feats),
], remainder='drop')

knn_params = len(num_feats) + len(cat_feats)
# Models 
knn = Pipeline([
    ('prep', prep_knn),
    ('model', KNeighborsRegressor(n_neighbors=knn_params, weights='distance', metric='minkowski')),
])

rf = Pipeline([
    ('prep', prep_rf),
    ('model', RandomForestRegressor(
        n_estimators=150,
        max_depth=20,
        min_samples_leaf=3,
        max_features='sqrt',
        random_state=42,
        n_jobs=-1,
    )),
])

# Fit
knn.fit(X_train, y_train)
rf.fit(X_train, y_train)

# Evaluate
def rmse_fn(y_true, y_pred):
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    return float(np.sqrt(np.mean((y_true - y_pred) ** 2)))

metrics = []
for name, est in [('KNN', knn), ('RandomForest', rf)]:
    pred = est.predict(X_test)
    r2 = r2_score(y_test, pred)
    rmse = rmse_fn(y_test, pred)
    metrics.append((name, r2, rmse))
    print(f'{name}: R2={r2:.3f} | RMSE={rmse:,.1f}')

# Helper to get transformed feature names from a fitted pipeline's 'prep'
def get_tnames(fitted_pipe: Pipeline, X_like: pd.DataFrame) -> np.ndarray:
    try:
        ct = fitted_pipe.named_steps['prep']
        names = ct.get_feature_names_out()
        # sanity-check: if length doesn't match the transformed shape, adjust
        n_out = ct.transform(X_like.iloc[:1]).shape[1]
        if len(names) != n_out:
            names = np.array([f'f{i}' for i in range(n_out)])
        return np.array([str(n) for n in names])
    except Exception:
        n_out = fitted_pipe.named_steps['prep'].transform(X_like.iloc[:1]).shape[1]
        return np.array([f'f{i}' for i in range(n_out)])

# Aggregate permutation importances back to base feature names (on a small X_test subset)
def aggregate_importance(estimator: Pipeline, X_df: pd.DataFrame, y_ser: pd.Series, n_repeats: int = 3, random_state: int = 42) -> pd.DataFrame:
    # Subsample test set for speed
    max_eval = 2000
    if len(X_df) > max_eval:
        X_eval = X_df.sample(n=max_eval, random_state=random_state)
        y_eval = y_ser.loc[X_eval.index]
    else:
        X_eval, y_eval = X_df, y_ser
    perm = permutation_importance(estimator, X_eval, y_eval, n_repeats=n_repeats, random_state=random_state, n_jobs=1)
    tnames = get_tnames(estimator, X_eval)
    n_imp = perm.importances_mean.shape[0]
    # Align lengths if needed
    if len(tnames) != n_imp:
        if len(tnames) > n_imp:
            tnames = tnames[:n_imp]
        else:
            tnames = np.concatenate([tnames, np.array([f'f{i}' for i in range(len(tnames), n_imp)])])
    imp = pd.DataFrame({'feature': tnames, 'importance': perm.importances_mean})
    imp['feature'] = imp['feature'].str.replace(r'^(num|cat)__', '', regex=True)
    # Map one-hot columns back to base categorical names
    def to_base(s: str) -> str:
        if not cat_feats:
            return s
        for c in cat_feats:
            if s.startswith(c + '_'):
                return c
        return s
    imp['base'] = imp['feature'].map(to_base)
    agg = imp.groupby('base', as_index=False)['importance'].sum().sort_values('importance', ascending=False)
    return agg

# Compute importances
try:
    imp_knn = aggregate_importance(knn, X_test, y_test)
except Exception as e:
    print('KNN permutation importance failed:', e)
    imp_knn = pd.DataFrame(columns=['base','importance'])

try:
    imp_rf = aggregate_importance(rf, X_test, y_test)
except Exception as e:
    print('RF permutation importance failed, falling back to impurity importances:', e)
    # Fallback: impurity importances mapped to base names
    try:
        rf_model = rf.named_steps['model']
        tnames = get_tnames(rf, X_test)
        fi = getattr(rf_model, 'feature_importances_', None)
        if fi is not None and len(fi) == len(tnames):
            tmp = pd.DataFrame({'feature': tnames, 'importance': fi})
            tmp['feature'] = tmp['feature'].str.replace(r'^(num|cat)__', '', regex=True)
            def to_base2(s: str) -> str:
                for c in cat_feats:
                    if s.startswith(c + '_'):
                        return c
                return s
            tmp['base'] = tmp['feature'].map(to_base2)
            imp_rf = tmp.groupby('base', as_index=False)['importance'].sum().sort_values('importance', ascending=False)
        else:
            imp_rf = pd.DataFrame(columns=['base','importance'])
    except Exception as e2:
        print('RF impurity importance fallback failed:', e2)
        imp_rf = pd.DataFrame(columns=['base','importance'])


KNN: R2=0.510 | RMSE=18,705.0
RandomForest: R2=0.738 | RMSE=13,675.6


In [9]:

print('\nTop 5 predictors — KNN:')
print(imp_knn.head(5))
print('\nTop 5 predictors — RandomForest:')
print(imp_rf.head(5))

# Report the single top predictor per model
top_knn = imp_knn.iloc[0]['base'] if not imp_knn.empty else None
top_rf  = imp_rf.iloc[0]['base'] if not imp_rf.empty else None

# Save all ranked predictors for both models to a single CSV
# Add rank column to all predictors
knn_all = imp_knn.copy()
knn_all = knn_all.reset_index(drop=True)
knn_all.insert(0, 'model', 'KNN')
knn_all.insert(1, 'rank', range(1, len(knn_all) + 1))
rf_all = imp_rf.copy()
rf_all = rf_all.reset_index(drop=True)
rf_all.insert(0, 'model', 'RandomForest')
rf_all.insert(1, 'rank', range(1, len(rf_all) + 1))
combined = pd.concat([
    knn_all[['model','rank','base','importance']],
    rf_all[['model','rank','base','importance']]
], ignore_index=True)
combined = combined.rename(columns={'base': 'feature'})
out_dir = Path('cleaned_data')
out_dir.mkdir(parents=True, exist_ok=True)
out_file = out_dir / 'top_predictors_rq1.csv'
combined.to_csv(out_file, index=False)
print('Saved all ranked predictors to', out_file)

# Choose best by RMSE
metrics_sorted = sorted(metrics, key=lambda x: x[2])
best_name = metrics_sorted[0][0]
best_cv_model = type('Obj', (), {})()
best_cv_model.best_estimator_ = knn if best_name == 'KNN' else rf
final_model_for_export = rf
print(f"\nSelected final model (by RMSE): {best_name}")
# After selecting final_model, generate predictions and save both listing-level and neighbourhood-level outputs
final_model = globals().get('final_model_for_export', None)
if final_model is None:
    final_model = best_cv_model.best_estimator_

full_preds = pd.Series(final_model.predict(features_df), index=features_df.index, name='predicted_revenue')

# Build listing-level output
o_cols = ['listing_id','city','month','neighbourhood','price','last_scraped','latitude','longitude',
          'estimated_revenue_l365d','estimated_occupancy_l365d','availability_365','availability_30',
          'reviews_per_month','number_of_reviews','amenities','property_type','room_type','accommodates',
          'bathrooms','bathrooms_text','bedrooms','beds','host_is_superhost','host_since','license',
          'calculated_host_listings_count','minimum_nights','maximum_nights']
# align o_cols with available in listings
o_cols = [c for c in o_cols if c in listings.columns]
out = listings[o_cols].copy()
out['predicted_revenue'] = full_preds.reindex(out.index)

# Save listing-level
cleaned_dir.mkdir(parents=True, exist_ok=True)
pred_path = cleaned_dir / 'predictions_rq1.csv'
out.to_csv(pred_path, index=False)
print('Saved listing-level predictions to', pred_path)

# Neighbourhood aggregation
neigh = neigh_col if 'neigh_col' in globals() else 'neighbourhood'
if neigh not in out.columns and 'neighbourhood' in out.columns:
    neigh = 'neighbourhood'
agg = out.groupby(['city', neigh], as_index=False)['predicted_revenue'].mean().rename(columns={neigh:'neighbourhood'})
agg_path = cleaned_dir / 'predictions_rq1_by_neighbourhood.csv'
agg.to_csv(agg_path, index=False)
print('Saved neighbourhood-level predictions to', agg_path)


Top 5 predictors — KNN:
                 base  importance
13  reviews_per_month    0.166347
12      property_type    0.159103
11              price    0.138044
9      minimum_nights    0.124665
1     amenities_count    0.087607

Top 5 predictors — RandomForest:
                 base  importance
11              price    0.426045
10  number_of_reviews    0.307581
13  reviews_per_month    0.231733
9      minimum_nights    0.148630
12      property_type    0.113577
Saved all ranked predictors to cleaned_data\top_predictors_rq1.csv

Selected final model (by RMSE): RandomForest
Saved listing-level predictions to cleaned_data\predictions_rq1.csv
Saved neighbourhood-level predictions to cleaned_data\predictions_rq1_by_neighbourhood.csv


## RQ2 — Compute ROI from predicted revenue
We merge the neighbourhood-level average predicted annual revenue with 2025 benchmark house prices to compute ROI = revenue ÷ price, then rank top neighbourhoods per city.

In [10]:
# RQ2 — ROI computation using predictions and 2025 benchmark prices with robust matching (token-based)

pred_path = Path('cleaned_data') / 'predictions_rq1_by_neighbourhood.csv'
assert pred_path.exists(), 'Run the RQ1 cell to generate neighbourhood-level predictions first.'

preds = pd.read_csv(pred_path)
# normalize column names
preds.columns = [c.strip() for c in preds.columns]
if 'neighbourhood' not in preds.columns:
    nh_cands = [c for c in preds.columns if 'neigh' in c.lower() or 'hood' in c.lower()]
    if nh_cands:
        preds.rename(columns={nh_cands[0]: 'neighbourhood'}, inplace=True)
if 'city' not in preds.columns:
    raise AssertionError('Predictions must contain a city column')
if 'predicted_revenue' not in preds.columns:
    alt = [c for c in preds.columns if 'pred' in c.lower() and 'rev' in c.lower()] 
    if alt:
        preds.rename(columns={alt[0]: 'predicted_revenue'}, inplace=True)

preds['neighbourhood'] = preds['neighbourhood'].astype(str).str.strip()

# Helper normalization for name matching
def normalize_name(s: str) -> str:
    s = (s or '').lower()
    s = s.replace('&', ' and ').replace('–', '-')
    s = re.sub(r'[^a-z0-9\- ]+', ' ', s)
    s = re.sub(r'\s+', ' ', s).strip()
    # collapse dashes
    s = s.replace('-', ' ')
    # drop common noise words
    s = re.sub(r'\b(neighbou?rhood|area|district|community|village|centre|center|city)\b', ' ', s)
    # expand/normalize
    s = re.sub(r'\bmt\b', 'mount', s)
    s = re.sub(r'\bst\.?\b', 'saint', s)
    s = re.sub(r'\b(ve|vw)\b', '', s)  # REBGV suffixes
    # directions
    s = re.sub(r'\b(north|south|east|west|central)\b', '', s)
    s = re.sub(r'\s+', ' ', s).strip()
    return s

# token-set similarity (like fuzzywuzzy token_set_ratio)
def token_set_similarity(a: str, b: str) -> float:
    ta = set(normalize_name(a).split())
    tb = set(normalize_name(b).split())
    if not ta or not tb:
        return 0.0
    inter = len(ta & tb)
    return (2.0 * inter) / (len(ta) + len(tb))

# Parse currency price strings robustly
_def_price_re = re.compile(r'[^0-9.]+')

def parse_price_str(x):
    if pd.isna(x):
        return pd.NA
    if isinstance(x, (int, float)):
        return float(x)
    s = str(x)
    s = _def_price_re.sub('', s)
    try:
        return float(s) if s else pd.NA
    except Exception:
        return pd.NA

# Helper to load and normalize a price CSV
def load_price_csv(path: Path, city_name: str) -> pd.DataFrame:
    df = pd.read_csv(path)
    df.columns = [c.strip() for c in df.columns]
    nh_cands = [c for c in df.columns if any(k in c.lower() for k in ['neigh', 'hood', 'area', 'district', 'community'])]
    if not nh_cands:
        first_non_num = next((c for c in df.columns if not pd.api.types.is_numeric_dtype(df[c])), None)
        if first_non_num is None:
            raise KeyError(f'Could not detect neighbourhood column in {path.name}')
        nh_col = first_non_num
    else:
        nh_pref = [c for c in nh_cands if 'neigh' in c.lower() or 'hood' in c.lower()]
        nh_col = nh_pref[0] if nh_pref else nh_cands[0]
    price_cands = [c for c in df.columns if any(k in c.lower() for k in ['price', 'avg', 'average', 'hpi', 'value', 'benchmark'])]
    price_cands = [c for c in price_cands if c != nh_col]
    if not price_cands:
        num_cols = [c for c in df.columns if pd.api.types.is_numeric_dtype(df[c])]
        if not num_cols:
            raise KeyError(f'Could not detect price column in {path.name}')
        price_col = num_cols[0]
    else:
        pr_pref = [c for c in price_cands if any(k in c.lower() for k in ['price','benchmark'])]
        price_col = pr_pref[0] if pr_pref else price_cands[0]

    out = df[[nh_col, price_col]].copy()
    out.columns = ['neighbourhood', 'benchmark_price']
    out['neighbourhood'] = out['neighbourhood'].astype(str).str.strip()
    # robust currency parsing
    out['benchmark_price'] = out['benchmark_price'].apply(parse_price_str)
    out['benchmark_price'] = pd.to_numeric(out['benchmark_price'], errors='coerce')
    out = out.dropna(subset=['benchmark_price'])
    out['city'] = city_name
    return out

van_prices = load_price_csv(Path('original_data') / '2025_van_neighbourhoods_price.csv', 'Vancouver')
vic_prices = load_price_csv(Path('original_data') / '2025_vic_neighbourhoods_price.csv', 'Victoria')
prices = pd.concat([van_prices, vic_prices], ignore_index=True)

# Create normalized keys
preds['key'] = preds['neighbourhood'].map(normalize_name)
prices['key'] = prices['neighbourhood'].map(normalize_name)

# Try exact normalized join first
merged = preds.merge(prices, on=['city','key'], how='inner', suffixes=('_pred','_price'))

# Build token-set fuzzy mapping for unmatched
matched_keys = set(merged['key'].unique()) if not merged.empty else set()
unmatched = preds[~preds['key'].isin(matched_keys)].copy()
rows = []
for city_name, grp in unmatched.groupby('city'):
    cand_df = prices[prices['city']==city_name]
    if cand_df.empty:
        continue
    cand_keys = cand_df['key'].unique().tolist()
    for _, r in grp[['neighbourhood','key']].drop_duplicates().iterrows():
        best_score, best_key = 0.0, None
        for ck in cand_keys:
            # quick filter: at least one shared token
            if not set(r['key'].split()) & set(ck.split()):
                continue
            s = token_set_similarity(r['key'], ck)
            if s > best_score:
                best_score, best_key = s, ck
        if best_key is not None and best_score >= 0.6:
            rows.append({'city': city_name, 'key': r['key'], 'key_price': best_key, 'match_score': best_score})

map_df = pd.DataFrame(rows)
if not map_df.empty:
    tmp = preds.merge(map_df[['city','key','key_price']], on=['city','key'], how='left')
    tmp['join_key'] = tmp['key_price'].fillna(tmp['key'])
    prices2 = prices.rename(columns={'key': 'join_key'})
    merged2 = tmp.merge(prices2, on=['city','join_key'], how='inner', suffixes=('_pred','_price'))
    if not merged.empty:
        existing = set(zip(merged['city'], merged['neighbourhood_pred']))
        merged2 = merged2[[ (c not in existing) for c in zip(merged2['city'], merged2['neighbourhood_pred']) ]]
        merged = pd.concat([merged, merged2], ignore_index=True)
    else:
        merged = merged2

# Save mapping debug
out_dir = Path('cleaned_data')
out_dir.mkdir(parents=True, exist_ok=True)
if not map_df.empty:
    map_df.to_csv(out_dir / 'roi_name_mapping.csv', index=False)
    print('Saved name mapping to', out_dir / 'roi_name_mapping.csv')

# Compute ROI if we have matches
if merged.empty:
    # emit some debug samples
    print('ROI merge empty. Debug samples:')
    print('Pred keys (first 15):', sorted(preds['key'].unique())[:15])
    print('Price keys Vancouver (first 15):', sorted(prices[prices['city']=='Vancouver']['key'].unique())[:15])
    print('Price keys Victoria (first 15):', sorted(prices[prices['city']=='Victoria']['key'].unique())[:15])
else:
    merged = merged.rename(columns={'neighbourhood_pred':'neighbourhood'})
    merged['predicted_revenue'] = pd.to_numeric(merged['predicted_revenue'], errors='coerce')
    merged = merged.dropna(subset=['predicted_revenue','benchmark_price'])
    merged['roi_ratio'] = merged['predicted_revenue'] / merged['benchmark_price']

    # Top 3 by city (handle <3 gracefully)
    top3 = merged.sort_values(['city','roi_ratio'], ascending=[True, False]).groupby('city').head(3)
    print('Top neighbourhoods by ROI (predicted_revenue / benchmark_price):')
    print(top3[['city','neighbourhood','predicted_revenue','benchmark_price','roi_ratio']])

    # Save outputs
    top3[top3['city']=='Vancouver'][['city','neighbourhood','predicted_revenue','benchmark_price','roi_ratio']].to_csv(out_dir / 'roi_vancouver.csv', index=False)
    top3[top3['city']=='Victoria'][['city','neighbourhood','predicted_revenue','benchmark_price','roi_ratio']].to_csv(out_dir / 'roi_victoria.csv', index=False)
    merged[['city','neighbourhood','predicted_revenue','benchmark_price','roi_ratio']].to_csv(out_dir / 'roi_merged_full.csv', index=False)
    print('Saved:', out_dir / 'roi_vancouver.csv')
    print('Saved:', out_dir / 'roi_victoria.csv')
    print('Saved full merge:', out_dir / 'roi_merged_full.csv')

Saved name mapping to cleaned_data\roi_name_mapping.csv
Top neighbourhoods by ROI (predicted_revenue / benchmark_price):
         city        neighbourhood  predicted_revenue  benchmark_price  \
33  Vancouver             Downtown       25337.342661         660017.0   
10  Vancouver           Strathcona       29079.250746         917658.0   
37  Vancouver  Renfrew-Collingwood       25968.415758         900658.0   
29   Victoria                Sooke       21406.648195         869400.0   
30   Victoria        Victoria West       20652.942473        1020900.0   
18   Victoria             Langford       20647.368235        1048000.0   

    roi_ratio  
33   0.038389  
10   0.031689  
37   0.028833  
29   0.024622  
30   0.020230  
18   0.019702  
Saved: cleaned_data\roi_vancouver.csv
Saved: cleaned_data\roi_victoria.csv
Saved full merge: cleaned_data\roi_merged_full.csv


## RQ3 — Tests (ANOVA first, then post hoc)
workflow:
1) Two-way ANOVA (City × Period) for average nightly price (listing-level).
2) If ANOVA shows significance, run Tukey HSD post hoc on City×Period groups.
3) Complementary checks: Welch t-tests (price Pre vs Post within each city), and Chi-square/proportion tests for licence share.
Period uses city-specific pre/post lists when available; otherwise month < 2025-05 is Pre.

In [11]:
# RQ3 : read cleaned_data/listings_unified_clean.csv and run tests
import numpy as np, pandas as pd, re
from pathlib import Path
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.proportion import proportions_ztest
from scipy.stats import ttest_ind, chi2_contingency

# Locate file robustly
root = Path.cwd()
csv_path = root / 'cleaned_data' / 'listings_unified_clean.csv'
if not csv_path.exists():
    # try parent
    csv_path = root.parent / 'cleaned_data' / 'listings_unified_clean.csv'
print(f'CSV path: {csv_path}')

# Output directory for results
out_dir = root / 'cleaned_data'
out_dir.mkdir(parents=True, exist_ok=True)

# Load
df = pd.read_csv(csv_path)

# Helper: find column by candidates (case-insensitive)
def find_col(cols, candidates):
    low = {c.lower(): c for c in cols}
    for name in candidates:
        for key, orig in low.items():
            if key == name.lower():
                return orig
    return None

def find_col_contains_numeric(df, substrings):
    for col in df.columns:
        if any(s in col.lower() for s in substrings):
            if pd.api.types.is_numeric_dtype(df[col]):
                return col
    return None

# Determine core columns
city_col = find_col(df.columns, ['city'])
month_col = find_col(df.columns, ['month','date','period_month','ym'])
period_col = find_col(df.columns, ['Period','period'])
price_col = find_col(df.columns, ['price']) or find_col_contains_numeric(df, ['price'])

if city_col is None or month_col is None or price_col is None:
    raise ValueError(f'Missing required columns. Found city={city_col}, month={month_col}, price={price_col}.')

# Normalize months to datetime for ordering and standardized string
mtry = pd.to_datetime(df[month_col], errors='coerce')
if mtry.isna().all():
    mtry = pd.to_datetime(df[month_col].astype(str).str[:7], errors='coerce')

# Standardize to first-of-month string for exact matching
month_std = mtry.dt.to_period('M').dt.to_timestamp()
month_str = month_std.dt.strftime('%Y-%m-01')

df['_City'] = df[city_col]
df['_MonthDT'] = month_std
df['_MonthStr'] = month_str

# Determine Period: prefer existing; else derive from known four months; else per-city median split
if period_col is not None:
    per = df[period_col].astype(str).str.strip().str.title()
    df['_Period'] = per.astype('object')
else:
    uniq = set(df['_MonthStr'].dropna().unique().tolist())
    known_four = {'2025-03-01','2025-04-01','2025-05-01','2025-06-01'}
    if uniq.issuperset(known_four):
        df['_Period'] = pd.Series(pd.NA, index=df.index, dtype='object')
        pre_set = {'2025-03-01','2025-04-01'}; post_set = {'2025-05-01','2025-06-01'}
        df.loc[df['_MonthStr'].isin(pre_set), '_Period'] = 'Pre'
        df.loc[df['_MonthStr'].isin(post_set), '_Period'] = 'Post'
        print('Assigned Period from month: Pre=[2025-03-01, 2025-04-01], Post=[2025-05-01, 2025-06-01].')
    else:
        df['_Period'] = pd.Series(pd.NA, index=df.index, dtype='object')
        for c, sub in df.groupby('_City'):
            dates = sub['_MonthDT'].dropna().sort_values().unique()
            if len(dates) == 0:
                continue
            cutoff = pd.Series(dates).median()
            df.loc[df['_City'].eq(c) & df['_MonthDT'].le(cutoff), '_Period'] = 'Pre'
            df.loc[df['_City'].eq(c) & df['_MonthDT'].gt(cutoff), '_Period'] = 'Post'
        print('Note: Period column missing and months not exactly the known four; used per-city median month split.')

# If no rows assigned to Pre/Post, fallback to median split
if df['_Period'].isin(['Pre','Post']).sum() == 0:
    df['_Period'] = pd.Series(pd.NA, index=df.index, dtype='object')
    for c, sub in df.groupby('_City'):
        dates = sub['_MonthDT'].dropna().sort_values().unique()
        if len(dates) == 0:
            continue
        cutoff = pd.Series(dates).median()
        df.loc[df['_City'].eq(c) & df['_MonthDT'].le(cutoff), '_Period'] = 'Pre'
        df.loc[df['_City'].eq(c) & df['_MonthDT'].gt(cutoff), '_Period'] = 'Post'
    print('Fallback applied: per-city median month split used because explicit mapping produced zero rows.')

# Keep only rows with defined Period and numeric price
work = df.loc[df['_Period'].isin(['Pre','Post']) & pd.notna(df[price_col])].copy()
work.rename(columns={price_col: '_Price'}, inplace=True)

print(f'Rows after cleaning: {len(work):,}; Cities={sorted(work._City.unique())}; Periods={work._Period.unique().tolist()}')

# Two-way ANOVA (Price) — City × Period
anova_path = out_dir / 'rq3_anova_price.csv'
tukey_path = out_dir / 'rq3_tukey_price.csv'
price_summary_path = out_dir / 'rq3_price_summary.csv'
welch_summary_path = out_dir / 'rq3_welch_price.csv'
license_share_path = out_dir / 'rq3_license_share.csv'
license_chi2_path = out_dir / 'rq3_license_chi2.csv'

if work._City.nunique() >= 2 and work._Period.nunique() >= 2:
    mdl = smf.ols('_Price ~ C(_City) * C(_Period)', data=work).fit()
    aov = sm.stats.anova_lm(mdl, typ=2)
    print('\nTwo-way ANOVA (Price) — City * Period')
    print(aov)
    aov.to_csv(anova_path, index=True)
    p_int = aov.loc['C(_City):C(_Period)', 'PR(>F)'] if 'C(_City):C(_Period)' in aov.index else np.nan
    do_tukey = (pd.notna(p_int) and p_int < 0.05)
    print(f"\nANOVA interaction (Price) p={p_int if pd.notna(p_int) else 'NA'}; Tukey HSD will run: {bool(do_tukey)}\n")
    if do_tukey:
        grp = work['_City'] + ' | ' + work['_Period']
        tuk = pairwise_tukeyhsd(work['_Price'].values, grp.values, alpha=0.05)
        print('Tukey HSD — Price by City * Period')
        print(tuk.summary())
        # Save Tukey as CSV — reconstruct pair labels in default order (i<j)
        groups = list(tuk.groupsunique)
        pairs = []
        for i in range(len(groups)-1):
            for j in range(i+1, len(groups)):
                pairs.append((groups[i], groups[j]))
        tuk_df = pd.DataFrame({
            'group1': [p[0] for p in pairs],
            'group2': [p[1] for p in pairs],
            'meandiff': tuk.meandiffs,
            'p_adj': tuk.pvalues,
            'lower': tuk.confint[:,0],
            'upper': tuk.confint[:,1],
            'reject': tuk.reject
        })
        tuk_df.to_csv(tukey_path, index=False)
else:
    print('\nANOVA skipped: need at least 2 cities and 2 periods with data.')

# Price summaries and Welch t-tests — Post vs Pre within each City
print('\nWelch t-tests — Price (Post vs Pre) within each City')
# Summary means/stats for plotting
price_summary = (
    work.groupby(['_City','_Period'])['_Price']
        .agg(mean='mean', std='std', n='count')
        .reset_index()
        .rename(columns={'_City':'City','_Period':'Period','mean':'mean_price','std':'std_price','n':'n'})
)
price_summary.to_csv(price_summary_path, index=False)

welch_rows = []
for c in sorted(work['_City'].unique()):
    pre_vals = work.loc[(work['_City']==c) & (work['_Period']=='Pre'), '_Price']
    post_vals = work.loc[(work['_City']==c) & (work['_Period']=='Post'), '_Price']
    m_pre, m_post = pre_vals.mean(), post_vals.mean()
    if pre_vals.count() >= 2 and post_vals.count() >= 2:
        t_stat, p_val = ttest_ind(post_vals, pre_vals, equal_var=False, nan_policy='omit')
        print(f"  {c}: t={t_stat:.3f}, p={p_val:.4g}, n_pre={pre_vals.count()}, n_post={post_vals.count()}")
        welch_rows.append({'City': c, 't': t_stat, 'p': p_val, 'n_pre': int(pre_vals.count()), 'n_post': int(post_vals.count()), 'mean_pre': m_pre, 'mean_post': m_post, 'diff_post_minus_pre': m_post - m_pre})
    else:
        print(f"  {c}: skipped (need ≥2 obs in both Pre and Post)")
welch_df = pd.DataFrame(welch_rows)
welch_df.to_csv(welch_summary_path, index=False)

# License tests — Chi-square and per-city proportion z-tests
lic_col = find_col(df.columns, ['Licensed','licensed','is_licensed','license','has_license'])
if lic_col is None:
    print('\nLicense tests skipped: no license indicator column found in CSV.')
else:
    pl = df.copy()
    pl['_City'] = pl['_City']
    pl['_Period'] = df['_Period']

    # Map license to binary using regex-friendly patterns
    def license_to_bin(v):
        if pd.isna(v):
            return 0
        s = str(v).strip()
        if not s:
            return 0
        s_low = s.lower()
        # obvious non-license notes
        if 'long term' in s_low or '90+' in s_low:
            return 0
        if s_low in {'am','pm'}:
            return 0
        # match license-like IDs: '25-156058' or 6+ digits anywhere in the string
        if re.search(r'\b\d{2}-\d{5,}\b', s):
            return 1
        if re.search(r'\b\d{6,}\b', s):
            return 1
        # common affirmative tokens
        if s_low in {'yes','y','true','t','licensed'}:
            return 1
        # explicit negatives
        if s_low in {'no','n','false','f','unlicensed'}:
            return 0
        return 0

    pl['_lic_bin'] = pl[lic_col].map(license_to_bin).astype(int)
    pl = pl.dropna(subset=['_City','_Period'])
    pl = pl[pl['_Period'].isin(['Pre','Post'])]

    if len(pl)==0 or pl['_lic_bin'].nunique()<2 or pl['_Period'].nunique()<2:
        print('\nChi-square skipped: not enough variation or missing Pre/Post after cleaning.')
    else:
        tab = pd.crosstab(pl['_Period'], pl['_lic_bin'])
        if tab.shape[0] >= 2 and tab.shape[1] >= 2:
            chi2, p, dof, exp = chi2_contingency(tab.values)
            print('\nChi-square — License by Period (overall):')
            print(tab)
            print(f"chi2={chi2:.3f}, dof={dof}, p={p:.4g}")
            pd.DataFrame([{'chi2': chi2, 'dof': dof, 'p': p}]).to_csv(license_chi2_path, index=False)
        else:
            print('\nChi-square skipped: contingency not at least 2x2 after cleaning.')

        # Per-city proportion z-tests — and save share summary for plots
        share_rows = []
        print('\nProportion z-tests — Licensed share (Post vs Pre) per City')
        for c in sorted(pl['_City'].unique()):
            sub = pl[pl['_City'].eq(c)]
            g = sub.groupby('_Period')['_lic_bin'].agg(['sum','count'])
            if {'Pre','Post'}.issubset(g.index):
                count = np.array([g.loc['Post','sum'], g.loc['Pre','sum']], dtype=float)
                nobs = np.array([g.loc['Post','count'], g.loc['Pre','count']], dtype=float)
                if nobs.min() > 0:
                    z, pz = proportions_ztest(count, nobs)
                    share_post = count[0]/nobs[0]; share_pre = count[1]/nobs[1]
                    print(f"  {c}: z={z:.3f}, p={pz:.4g}, post={share_post:.3f}, pre={share_pre:.3f}, n_post={nobs[0]:.0f}, n_pre={nobs[1]:.0f}")
                    share_rows.append({'City': c, 'Period': 'Pre',  'licensed_share': share_pre,  'licensed_n': int(count[1]), 'n': int(nobs[1])})
                    share_rows.append({'City': c, 'Period': 'Post', 'licensed_share': share_post, 'licensed_n': int(count[0]), 'n': int(nobs[0])})
                else:
                    print(f"  {c}: skipped (zero observations in Pre or Post)")
            else:
                print(f"  {c}: skipped (missing Pre or Post)")
        share_df = pd.DataFrame(share_rows)
        share_df.to_csv(license_share_path, index=False)

print(f"\nSaved: {price_summary_path.name}, {welch_summary_path.name}, {anova_path.name}, {tukey_path.name if (out_dir/tukey_path.name).exists() else 'tukey (none)'}, {license_share_path.name} (if license present), {license_chi2_path.name} (if ran)")

CSV path: c:\Users\alext\OneDrive\桌面\CMPT 353\Project\cleaned_data\listings_unified_clean.csv
Assigned Period from month: Pre=[2025-03-01, 2025-04-01], Post=[2025-05-01, 2025-06-01].
Rows after cleaning: 14,479; Cities=['Vancouver', 'Victoria']; Periods=['Pre', 'Post']

Two-way ANOVA (Price) — City * Period
                           sum_sq       df          F        PR(>F)
C(_City)             7.690041e+05      1.0  14.665230  1.289343e-04
C(_Period)           4.698395e+06      1.0  89.600365  3.356588e-21
C(_City):C(_Period)  4.364934e+05      1.0   8.324112  3.918001e-03
Residual             7.590290e+08  14475.0        NaN           NaN

ANOVA interaction (Price) p=0.003918001138383897; Tukey HSD will run: True

Tukey HSD — Price by City * Period
           Multiple Comparison of Means - Tukey HSD, FWER=0.05           
     group1           group2     meandiff p-adj   lower    upper   reject
-------------------------------------------------------------------------
Vancouver | Post 